## Work done by:

#### <b><i>Cham An Fam</i></b> | <b><i>Simon Idoko</i></b> | <b><i>Ruslan Damindarov</i></b>

<h2>Polar Robot</h2>
Kinematics using Product of Exponentials

### <b><i>Imports</i></b>

In [1]:
import numpy as np
import sympy as sym
from functools import reduce

### <b><i>Utilities</i></b>

In [2]:
def Rx(q):
  T = np.array([[1,         0,          0, 0],
                [0, np.cos(q), -np.sin(q), 0],
                [0, np.sin(q),  np.cos(q), 0],
                [0,         0,          0, 1]], dtype=float)
  return T

def Ry(q):
  T = np.array([[ np.cos(q), 0, np.sin(q), 0],
                [         0, 1,         0, 0],
                [-np.sin(q), 0, np.cos(q), 0],
                [         0, 0,         0, 1]], dtype=float)
  return T

def Rz(q):
  T = np.array([[np.cos(q), -np.sin(q), 0, 0],
                [np.sin(q),  np.cos(q), 0, 0],
                [        0,          0, 1, 0],
                [        0,          0, 0, 1]], dtype=float)
  return T

In [3]:
def Tx(x):
  T = np.array([[1, 0, 0, x],
                [0, 1, 0, 0],
                [0, 0, 1, 0],
                [0, 0, 0, 1]], dtype=float)
  return T

def Ty(y):
  T = np.array([[1, 0, 0, 0],
                [0, 1, 0, y],
                [0, 0, 1, 0],
                [0, 0, 0, 1]], dtype=float)
  return T

def Tz(z):
  T = np.array([[1, 0, 0, 0],
                [0, 1, 0, 0],
                [0, 0, 1, z],
                [0, 0, 0, 1]], dtype=float)
  return T

In [4]:
def Tx_sym(s):
  return sym.Matrix(
      [[1, 0, 0, s],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]]
  )

def Ty_sym(s):
  return sym.Matrix(
      [[1, 0, 0, 0],
        [0, 1, 0, s],
        [0, 0, 1, 0],
        [0, 0, 0, 1]]
  )

def Tz_sym(s):
  return sym.Matrix(
      [[1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, s],
        [0, 0, 0, 1]]
  )

In [5]:
def Rx_sym(q):
  return sym.Matrix(
      [[1, 0, 0, 0],
        [0, sym.cos(q), -sym.sin(q), 0],
        [0, sym.sin(q), sym.cos(q), 0],
        [0, 0, 0, 1]]
  )

def Ry_sym(q):
  return sym.Matrix(
      [[sym.cos(q), 0, sym.sin(q), 0],
        [0, 1, 0, 0],
        [-sym.sin(q), 0, sym.cos(q), 0],
        [0, 0, 0, 1]]
  )

def Rz_sym(q):
  return sym.Matrix(
      [[sym.cos(q), -sym.sin(q), 0, 0],
        [sym.sin(q), sym.cos(q), 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]]
  )

In [6]:
def vec_cross(vec1, vec2):
  """Returns the cross product of two vectors"""
  return np.cross(np.aray(vec1), np.array(vec2))

def vec2skew(vec):
  """Takes a 3 dimensional vector and returns 
     the skew matrix representation of it"""
  x, y, z = np.asarray(vec)
  skew = np.array([
    [ 0, -z,  y],
    [ z,  0, -x],
    [-y,  x,  0]
  ])

  return skew

def T2Tadj(t_mat):
  """Takes the transformation matrix and returns 
     its adjoint representation"""
  R = t_mat[:3,:3] # Rotation part of T
  p = t_mat[:3,-1] # Translation part of T
    
  return np.block([
    [ R,             np.zeros_like(R) ],
    [ vec2skew(p)@R, R                ]
  ])

def twist2mat(twist):
  """Takes the 6 dimensional twist vector and returns 
     its 4X4 {se(3)} matrix representation"""
  ω = twist[:3]
  υ = twist[3:]

  return np.block([
      [ vec2skew(ω), np.c_[υ] ],
      [ np.zeros(3), 0        ]
  ])

In [7]:
def rot_exp(ω, θ):
  """Calculates the exponential matrix of the given rotation 
     matrix for a given angular velocity vector and theta value"""
  value = np.eye(3)\
          + np.sin(θ)*vec2skew(ω)\
          + (1-np.cos(θ))*vec2skew(ω)@vec2skew(ω)

  return value

def screw_exp(screw, θ):
  """Calculates the exponential matrix of the given 6 dimension screw 
     vector for a given theta value"""
  Sω = screw[:3]
  Sυ = screw[3:]

  if np.linalg.norm(Sω)==0 and np.linalg.norm(Sυ)==1:
    """Purely translational motion"""
    return np.block([
        [ np.eye(3),   np.c_[Sυ*θ] ],
        [ np.zeros(3), 1           ]
    ])
  elif np.linalg.norm(Sω)==1:
    """Screw axis motion with rotation"""
    exp12 = ((np.eye(3)*θ)\
            + (1-np.cos(θ))*vec2skew(Sω)\
            + (θ-np.sin(θ))*vec2skew(Sω)@vec2skew(Sω)) @ Sυ
    
    return np.block([
        [ rot_exp(Sω, θ),  np.c_[exp12] ],
        [ np.zeros(3),     1            ]
    ])
  else:
    print(f'Check screw vector: {screw}')

In [8]:
def mag(vec):
  """Returns the magnitude of a vector"""
  return np.linalg.norm(np.asarray(vec))

def RRx(q):
  R = np.array([[1,         0,          0],
                [0, np.cos(q), -np.sin(q)],
                [0, np.sin(q),  np.cos(q)]], dtype=float)
  return R

def RRy(q):
  R = np.array([[ np.cos(q), 0, np.sin(q)],
                [         0, 1,         0],
                [-np.sin(q), 0, np.cos(q)]], dtype=float)
  return R

def RRz(q):
  R = np.array([[np.cos(q), -np.sin(q), 0],
                [np.sin(q),  np.cos(q), 0],
                [        0,          0, 1]], dtype=float)
  return R

def rotate_xyz(vec, angx, angy, angz):
  return np.round( RRx(angx) @ RRy(angy) @ RRz(angz) @ np.asarray(vec), 2 )

### <b><i>Parameters And Product of Exponential Terms</i></b> 

In [9]:
# Parameter Definition
a1, a2, a3 = 0.2, 0.1, 0.5

# Definition of M and Screw Axis 
M = np.asarray([
      [ 1, 0, 0,     0 ],
      [ 0, 1, 0,    a3 ],
      [ 0, 0, 1, a1+a2 ],
      [ 0, 0, 0,     1 ]
])

s1 = np.array([ 0, 0, 1, 0,   0, 0])
s2 = np.array([ 1, 0, 0, 0,  a1, 0])
s3 = np.array([ 0, 0, 0, 0,   1, 0])
 
b1 = np.array([ 0, 0, 1, a3,   0,  0])
b2 = np.array([ 1, 0, 0,  0, -a2, a3])
b3 = np.array([ 0, 0, 0,  0,   1,  0])

### <b><i>Forward Kinematics</i></b> 

In [10]:
def FK(thetas):
  """Returns the Forward Transformation Matrix for given joint variables"""
  theta1, theta2, theta3 = thetas

  T_theta1 = screw_exp(s1, theta1) @ screw_exp(s2, theta2) @ screw_exp(s3, theta3) @ M
  T_theta2 = M @ screw_exp(b1, theta1) @ screw_exp(b2, theta2) @ screw_exp(b3, theta3)

  return np.round(T_theta1,4)

In [11]:
def classical_FK(thetas):
  theta1, theta2, theta3 = thetas

  FK = Rz(theta1) @ Tz(a1) @ Rx(theta2) @ Tz(a2) @ Ty(a3+theta3)

  return np.round(FK,4)

In [12]:
# Should return Zero Configuration Transformation Matrix
FK([0, 0, 0]) == classical_FK([0, 0, 0])

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

In [13]:
FK([-np.pi/2, 0, 1]) == classical_FK([-np.pi/2, 0, 1])

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

In [14]:
FK([-np.pi/2, 0.9, 0.6]) == classical_FK([-np.pi/2, 0.9, 0.6])

array([[ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True],
       [ True,  True,  True,  True]])

### <b><i>Jacobian</i></b> 

In [15]:
def jacobian_space(screws, thetas):
  
  J = np.array([])

  for (i,theta) in enumerate(thetas):

    s_exps = [ screw_exp(screws[i], thetas[i]) for i in range(i) ]  # List of all screw exponentials till before joint i
    s_exps.append(np.eye(4))                                        # To make sure it is not empty
    
    T = reduce(lambda a,b: a@b, s_exps)                             # Dot product of all screw_exponentials
    T_adj = T2Tadj(T)                                               # Adjoint of total screw_exponentials
    j_theta = T_adj @ screws[i]                                     # joint i jacobian vector

    if len(J)==0:
      J = np.c_[j_theta]
    else:
      J = np.c_[J, j_theta]
  
  return np.asarray(J).round(4)

In [16]:
def jacobian_body(bodys, thetas):
  
  J = np.array([])
   
  n = len(thetas)-1
  for (i,theta) in enumerate(thetas):

    s_exps = [ screw_exp(-bodys[i], thetas[i]) for i in np.arange(i, len(thetas))[::-1] ]  # List of all screw exponentials from joint 2 to joint i
    s_exps.append(np.eye(4))                                            # To make sure it is not empty
    # s_exps[::-1]

    T = reduce(lambda a,b: a@b, s_exps)                                 # Dot product of all screw_exponentials
    T_adj = T2Tadj(T)                                                   # Adjoint of total screw_exponentials
    j_theta = T_adj @ bodys[i]                                          # joint i jacobian vector

    if len(J)==0:
      J = np.c_[j_theta]
    else:
      J = np.c_[J, j_theta]
  
  return np.asarray(J).round(4)

In [17]:
def jacobian_sym():
  q1, q2, q3 = sym.symbols("q_1 q_2 q_3", real=True)  

  variables = [q1, q2, q3]

  TF =  Rz_sym(q1) @ Tz_sym(a1) @ \
        Rx_sym(q2) @ Tz_sym(a2) @ \
        Ty_sym(a3 + q3) 

  R = TF[:3,:-1]
  jacobian = sym.Matrix([])

  for var in variables:
      T_d  = sym.diff(TF, var) 

      T    = T_d[0:3, -1]
      R_d  = T_d[0:3, :-1]
      R_j  = R_d @ R.T 

      J = T.row_insert(3, sym.Matrix([R_j[2,1], R_j[0,2], R_j[1,0]]))

      jacobian = jacobian.col_insert(len(jacobian), J)

  return sym.lambdify([variables], jacobian, "numpy")

jacobian_sym_func = jacobian_sym()

def classical_jacobian(thetas):
  variables = [*thetas]
  return jacobian_sym_func(variables).round(4)

### <b><i>Comparing Jacobians</i></b> 

In [18]:
screws = [s1, s2, s3]
bodys  = [b1, b2, b3]
thetas = np.array([-np.pi/4, 4*np.pi/3, 1])

In [19]:
print("Jacobian Spacial")
jacobian_space(screws, thetas)

Jacobian Spacial


array([[ 0.    ,  0.7071,  0.    ],
       [ 0.    , -0.7071,  0.    ],
       [ 1.    ,  0.    ,  0.    ],
       [ 0.    ,  0.1414, -0.3536],
       [ 0.    ,  0.1414, -0.3536],
       [ 0.    ,  0.    , -0.866 ]])

In [20]:
print("Jacobian Body")
jacobian_body(bodys, thetas)

Jacobian Body


array([[ 0.    ,  1.    ,  0.    ],
       [-0.866 ,  0.    ,  0.    ],
       [-0.5   ,  0.    ,  0.    ],
       [ 1.6634,  0.    ,  0.    ],
       [ 0.    , -0.1   ,  1.    ],
       [ 0.    ,  1.5   ,  0.    ]])

In [21]:
print("Classical Jacobian")
classical_jacobian(thetas)

Classical Jacobian


array([[ 0.4691,  0.9539, -0.3536],
       [-0.4691,  0.9539, -0.3536],
       [ 0.    , -0.6634, -0.866 ],
       [ 0.    ,  0.7071,  0.    ],
       [ 0.    , -0.7071,  0.    ],
       [ 1.    ,  0.    ,  0.    ]])

### <b><i>Plot Function</i></b>

In [22]:
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go

In [23]:
def vec2euler(vec):
  x, y, z = np.asarray(vec)
  
  rx, ry, rz = np.arctan2(y,x), np.arctan2(x,y), np.arctan2(y,z)

  return [rx, ry, rz]

In [24]:
def to_frame(frame, vec):
  """"""
  vec = np.asarray(vec)
  frame = np.asarray(frame)

  x, y, z = frame 
  rx, ry, rz = vec2euler(frame)

  R = RRx(rx)@RRy(ry)@RRz(rz)

  R = R.T

  return (-R@frame + R@vec).round(4)

In [25]:
def box_3D(p0=[0,0,0], p1=[1,1,1], breadth=1, width=1):
  """Draws a solid box of certain height and width with its base centred at pos"""

  # convert to numpy arrays 
  p0, p1 = np.asarray(p0), np.asarray(p1)

  # vector in direction of axis
  v = p1 - p0

  # height along the axis
  h = mag(v)

  cx, cy, cz = p0
  x1, x2 = cx-breadth/2, cx+breadth/2
  y1, y2 = cy-width/2, cy+width/2
  z1, z2 = cz, cz+h

  xv, yv, zv = v
  angx, angy, angz = np.arctan2(yv,xv), np.arctan2(xv,yv), np.arctan2(yv,zv)

  print(np.rad2deg([angx, angy, angz]))

  # Rotate all points
  p111 = to_frame( -p0, ( rotate_xyz(to_frame( p0, [x1, y1, z1] ), angx, angy, angz) ) )
  p121 = to_frame( -p0, ( rotate_xyz(to_frame( p0, [x1, y2, z1] ), angx, angy, angz) ) )
  p221 = to_frame( -p0, ( rotate_xyz(to_frame( p0, [x2, y2, z1] ), angx, angy, angz) ) )
  p211 = to_frame( -p0, ( rotate_xyz(to_frame( p0, [x2, y1, z1] ), angx, angy, angz) ) )
  p112 = to_frame( -p0, ( rotate_xyz(to_frame( p0, [x1, y1, z2] ), angx, angy, angz) ) )
  p122 = to_frame( -p0, ( rotate_xyz(to_frame( p0, [x1, y2, z2] ), angx, angy, angz) ) )
  p222 = to_frame( -p0, ( rotate_xyz(to_frame( p0, [x2, y2, z2] ), angx, angy, angz) ) )
  p212 = to_frame( -p0, ( rotate_xyz(to_frame( p0, [x2, y1, z2] ), angx, angy, angz) ) )

  box = go.Mesh3d(
    # 8 vertices of a cube
    x=[p111[0], p121[0], p221[0], p211[0], p112[0], p122[0], p222[0], p212[0]],
    y=[p111[1], p121[1], p221[1], p211[1], p112[1], p122[1], p222[1], p212[1]],
    z=[p111[2], p121[2], p221[2], p211[2], p112[2], p122[2], p222[2], p212[2]],

    i = [7, 0, 0, 0, 4, 4, 6, 6, 4, 0, 3, 2],
    j = [3, 4, 1, 2, 5, 6, 5, 2, 0, 1, 6, 3],
    k = [0, 7, 2, 3, 6, 7, 1, 1, 5, 5, 7, 6],
    opacity=1,
    color='#DC143C',
    flatshading = True
  )

  return box

In [26]:
def cylinder3D(p0=[0,0,0], p1=[1,1,1], R=1):

  color_style = "Peach"

  p0 = np.asarray(p0)
  p1 = np.asarray(p1)

  # vector in direction of axis
  v = p1 - p0

  # find magnitude of vector
  mag = np.linalg.norm(v)

  #unit vector in direction of axis
  v = v / mag

  # make some vector not in the same direction as v
  not_v = np.array([1, 0, 0])
  if (np.abs(v) == np.abs(not_v)).all():
      not_v = np.array([0, 1, 0])

  # make vector perpendicular to v
  n1 = np.cross(v, not_v)

  # normalize n1
  n1 /= np.linalg.norm(n1)

  # make unit vector perpendicular to v and n1
  n2 = np.cross(v, n1)

  # surface ranges over t from 0 to length of axis and 0 to 2*pi
  t = np.linspace(0, mag, 2)
  theta = np.linspace(0, 2 * np.pi, 100)
  rsample = np.linspace(0, R, 2)

  # use meshgrid to make 2d arrays
  t, theta2 = np.meshgrid(t, theta)

  rsample,theta = np.meshgrid(rsample, theta)

  # generate coordinates for surface
  # "Tube"
  X, Y, Z = [p0[i] + v[i] * t + R * np.sin(theta2) * n1[i] + R * np.cos(theta2) *       n2[i] for i in [0, 1, 2]]
  # "Bottom"
  X2, Y2, Z2 = [p0[i] + rsample[i] * np.sin(theta) * n1[i] + rsample[i] * np.cos(theta) * n2[i] for i in [0, 1, 2]]
  # "Top"
  X3, Y3, Z3 = [p0[i] + v[i]*mag + rsample[i] * np.sin(theta) * n1[i] + rsample[i] * np.cos(theta) * n2[i] for i in [0, 1, 2]]

  tube = go.Surface(x=X, y=Y, z=Z, colorscale=color_style, showscale=False)
  bottom=dict(type='surface', x=X2, y=Y2, z=Z2, colorscale=color_style, showscale=False)
  top=dict(type='surface', x=X3, y=Y3, z=Z3, colorscale=color_style, showscale=False)

  return [tube, bottom, top]